In [1]:
import numpy as np
import pandas as pd
import re
from collections import defaultdict
from collections import Counter
import json
import string
from matplotlib import pyplot  as plt

### important links
* https://github.com/niklasstoehr/thesis/blob/18b18d00fcf8673aeac14293ccc62ab6f0e27173/4_real_attr/preprocess_amazon.ipynb
* https://github.com/sivapanuganti/Social-Network-Analysis
* https://github.com/Zhaomin-W/Network-based-Recommender
* https://github.com/ITWSDataScience/AmazonCopurchaseAnalysis2019
* https://advaitiyer.github.io/dsml/2020-01-03-adm/
* https://github.com/Leah-Wu/SI650/tree/main/code
### project paper link
link to the paper
* http://snap.stanford.edu/class/cs224w-2017/projects/cs224w-58-final.pdf

# Data Processing

#### sample text data 
* Id:   244410
* ASIN: B000031WDY
* title: Verehrt Und Angespien (Worshipped & Spat At)
* group: Music
* salesrank: 68859
* similar: 5  B00000JPCL  B0002CHIY2  B000066JGH  B0002XDODU  B0001CCY88
* categories: 3
* |Music[5174]|Styles[301668]|Rock[40]|General[67206]
* |Music[5174]|Styles[301668]|Hard Rock & Metal[67207]|General[67212]
* |Music[5174]|Specialty Stores[468040]|Indie Music[266023]|Rock[171251]|Hard Rock & Metal[520320]|Metal[520324]
* reviews: total: 8  downloaded: 8  avg rating: 5
* 1999-10-24  cutomer:  ATVPDKIKX0DER  rating: 5  votes:  12  helpful:  11
* 2000-4-15  cutomer: A3SPJZZZ64WYYU  rating: 4  votes:   8  helpful:   3
* 2001-4-2  cutomer: A2740NDTSIVCRB  rating: 5  votes:   3  helpful:   3
* 2002-4-8  cutomer: A1OA7C1729ANJS  rating: 5  votes:  10  helpful:  10
* 2003-7-4  cutomer: A26DVGNAARIU58  rating: 5  votes:   2  helpful:   2
* 2004-9-14  cutomer: A26D0985UODOGL  rating: 5  votes:   0  helpful:   0
* 2004-10-5  cutomer: A2A7C0MI0E91EB  rating: 5  votes:   0  helpful:   0
* 2005-3-1  cutomer: A1YGNYIDTG2QWA  rating: 4  votes:   0  helpful:   0

#### explaining data attributes:

* Id: Product id (number 0, ..., 548551)
* ASIN: Amazon Standard Identification Number
* title: Name/title of the product
* group: Product group (Book, DVD, Video or Music)
* salesrank: Amazon Salesrank
* co-purchased: ASINs of co-purchased products (people who buy X also buy Y)
* categories: Location in product category hierarchy to which the product belongs (separated by |, category id in [])
* reviews: Product review information: time, user id, rating, total number of votes on the review, total number of helpfulness votes (how many people found the review to be helpful)

In [2]:
%%time
# open file to read amazon product metadata 
fhr = open('data/amazon_data_file.txt', 'r', encoding='utf-8', errors='ignore')

# initialize a nested product dictionary that will hold cleaned up amazon product data
# key = ASIN; value = MetaData associated with ASIN
amazonProducts = {}

# read the data from the amazon-meta file;
# populate amazonProducts nested dicitonary;
ct=0

(Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating) = ("", "", "", list(), "", list(), 0, 0, 0.0)
(time_stamp,review_ASINs,review_ratings,review_votes,review_helpfull) = (list(),list(),list(),list(),list())
for line in fhr:
    if ct % 1000000 == 0:
        print("{} Done!".format(ct))
    ct+=1
    
    line = line.strip()
    # a product block started
    if(line.startswith("Id")):

        line = line.replace(" ","")
        Id = line.split(":")[1]
    
    elif(line.startswith("ASIN")):

        line = line.replace(" ","")
        ASIN = line.split(":")[1]
     
    elif(line.startswith("title")):
 
        Title = line.split(":")[1]
  
        Title = ' '.join(Title.split())
    elif(line.startswith("group")):
        Group = line.split(":")[1]
 
    elif(line.startswith("salesrank")):
        line = line.replace(" ","")
        SalesRank = line.split(":")[1]

    elif(line.startswith("similar")):
        ls = line.split()
        Copurchased = ([c for c in ls[2:]])
        
    elif(line.startswith("categories")):
        ls = line.split()
        cat_list = []
        for i in range(int(ls[1].strip())):
            temp = fhr.readline()
            temp = temp.lower()
            temp = temp.replace("|","-")
            temp = temp[4:]
            temp = temp.strip()
            cat_list.append(temp)
        
        Categories = cat_list

    elif(line.startswith("reviews")):
        ls = line.split()
        TotalReviews = ls[2].strip()
        AvgRating = ls[7].strip()
        l1,l2,l3,l4,l5=[],[],[],[],[]
        for i in range(int(ls[4].strip())):
            temp = fhr.readline()
            temp = temp.split()
            l1.append(temp[0])
            l2.append(temp[2])
            l3.append(temp[4])
            l4.append(temp[6])
            l5.append(temp[8])
        (time_stamp,review_ASINs,review_ratings,review_votes,review_helpfull)=(l1,l2,l3,l4,l5)
        
            
    # a product block ended
    # write out fields to amazonProducts Dictionary
    elif (line==""):
        try:
            MetaData = {}
            if (ASIN != ""):
                amazonProducts[ASIN]=MetaData
            MetaData['Id'] = Id            
            MetaData['title'] = Title
            MetaData['categories'] = Categories
            MetaData['group'] = Group
            MetaData['co-purchased'] = Copurchased
            MetaData['sales_rank'] = int(SalesRank)
            MetaData['total_reviews'] = int(TotalReviews)
            MetaData['avg_rating'] = float(AvgRating)
            MetaData['review_time_stamps_list']    = time_stamp
            MetaData['review_ASINs_list']          = review_ASINs
            MetaData['review_ratings_list']        = review_ratings
            MetaData['review_votes_list']          = review_votes
            MetaData['review_found_helpfull_list'] = review_helpfull
            
            
        except NameError:
            continue
        (Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating) = ("", "", "", list(), "", list(), 0, 0,0.0)
        (time_stamp,review_ASINs,review_ratings,review_votes,review_helpfull) = (list(),list(),list(),list(),list())
fhr.close()

0 Done!
1000000 Done!
2000000 Done!
3000000 Done!
4000000 Done!
Wall time: 30.8 s


In [4]:
%%time
df = pd.DataFrame(amazonProducts)
df = df.transpose()
temp = list(df['Id'].values)
df['Id'] = df.index
df.rename(columns={"Id":"ASIN"},inplace=True)
df.index = temp
new_col = {"ASIN":"product_id","review_ASINs_list":"user_ids"}
df = df.rename(new_col,axis=1)
df.head()

Wall time: 29.4 s


,product_id,title,categories,group,co-purchased,sales_rank,total_reviews,avg_rating,review_time_stamps_list,user_ids,review_ratings_list,review_votes_list,review_found_helpfull_list
0,0771044445,,[],,[],0,0,0.0,[],[],[],[],[]
1,0827229534,Patterns of Preaching,[books[283155]-subjects[1000]-religion & spiri...,Book,"[0804215715, 156101074X, 0687023955, 068707423...",396585,2,5.0,"[2000-7-28, 2003-12-14]","[A2JW67OY8U6HHK, A2VE83MZF98ITY]","[5, 5]","[10, 6]","[9, 5]"
2,0738700797,Candlemas,[books[283155]-subjects[1000]-religion & spiri...,Book,"[0738700827, 1567184960, 1567182836, 073870052...",168596,12,4.5,"[2001-12-16, 2002-1-7, 2002-1-24, 2002-1-28, 2...","[A11NCO6YTE4BTJ, A9CQ3PLRNIR83, A13SG9ACZ9O5IM...","[5, 4, 5, 5, 4, 4, 4, 5, 5, 5, 1, 5]","[5, 5, 8, 4, 16, 5, 6, 8, 8, 5, 13, 1]","[4, 5, 8, 4, 16, 5, 6, 8, 5, 5, 9, 1]"
3,0486287785,World War II Allied Fighter Planes Trading Cards,[books[283155]-subjects[1000]-home & garden[48...,Book,[],1270652,1,5.0,[2003-7-10],[A3IDGASRQAW8B2],[5],[2],[2]
4,0842328327,Life Application Bible Commentary,[books[283155]-subjects[1000]-religion & spiri...,Book,"[0842328130, 0830818138, 0842330313, 084232861...",631289,1,4.0,[2004-8-19],[A2591BUPXCS705],[4],[1],[1]


In [5]:
%%time
def parse_date(date_list):
    new_list=[]
    for date in date_list:
        x = date.split("-")
        year,month,day = int(x[0]),int(x[1]),int(x[2])
        new_list.append((year,month,day))
    return new_list

def clean_category(cat_id_list):
    new_list=[]
    for row  in cat_id_list:
        temp=[]
        x = row.split("-")
        for cid in x:
            if "[" in cid:
                y = cid.split("[")
                cat  = y[0]
                ID  = (y[1][:-1])
            else:
                cat = cid
                ID = -1
            temp.append((cat,ID))
        new_list.append(temp)
    return new_list       

df['product_id']  = df['product_id'].astype('str')
df['title'] = df['title'].apply(lambda x:re.sub(r'[^a-zA-Z0-9 ]','',x))
df['title'] = df['title'].str.lower()
df['categories'] = df['categories'].apply(clean_category)
df['group'] = df['group'].str.lower()
df['group'] = df['group'].str.replace(' ','')
df['group']  = df['group'].astype('str')
df['co-purchased']= df['co-purchased'].apply(lambda x: [str(i) for i in x])
df['sales_rank'] = df['sales_rank'].astype('int')
df['total_reviews'] = df['total_reviews'].astype('int')
df['avg_rating'] = df['avg_rating'].astype('float')
df['review_time_stamps_list'] = df['review_time_stamps_list'].apply(parse_date)
df['user_ids']= df['user_ids'].apply(lambda x: [str(i) for i in x])
df['review_ratings_list']= df['review_ratings_list'].apply(lambda x: [int(i) for i in x])
df['review_votes_list']= df['review_votes_list'].apply(lambda x: [int(i) for i in x])
df['review_found_helpfull_list']= df['review_found_helpfull_list'].apply(lambda x: [int(i) for i in x])
df['num_reviews'] = df['review_ratings_list'].apply(lambda x:len(x))
df.head()

Wall time: 44.4 s


,product_id,title,categories,group,co-purchased,sales_rank,total_reviews,avg_rating,review_time_stamps_list,user_ids,review_ratings_list,review_votes_list,review_found_helpfull_list,num_reviews
0,0771044445,,[],,[],0,0,0.0,[],[],[],[],[],0
1,0827229534,patterns of preaching,"[[(books, 283155), (subjects, 1000), (religion...",book,"[0804215715, 156101074X, 0687023955, 068707423...",396585,2,5.0,"[(2000, 7, 28), (2003, 12, 14)]","[A2JW67OY8U6HHK, A2VE83MZF98ITY]","[5, 5]","[10, 6]","[9, 5]",2
2,0738700797,candlemas,"[[(books, 283155), (subjects, 1000), (religion...",book,"[0738700827, 1567184960, 1567182836, 073870052...",168596,12,4.5,"[(2001, 12, 16), (2002, 1, 7), (2002, 1, 24), ...","[A11NCO6YTE4BTJ, A9CQ3PLRNIR83, A13SG9ACZ9O5IM...","[5, 4, 5, 5, 4, 4, 4, 5, 5, 5, 1, 5]","[5, 5, 8, 4, 16, 5, 6, 8, 8, 5, 13, 1]","[4, 5, 8, 4, 16, 5, 6, 8, 5, 5, 9, 1]",12
3,0486287785,world war ii allied fighter planes trading cards,"[[(books, 283155), (subjects, 1000), (home & g...",book,[],1270652,1,5.0,"[(2003, 7, 10)]",[A3IDGASRQAW8B2],[5],[2],[2],1
4,0842328327,life application bible commentary,"[[(books, 283155), (subjects, 1000), (religion...",book,"[0842328130, 0830818138, 0842330313, 084232861...",631289,1,4.0,"[(2004, 8, 19)]",[A2591BUPXCS705],[4],[1],[1],1


## EDA

In [6]:
%%time
total_userid = df['user_ids'].values
userids=set()
for row in total_userid:
    for ele in row:
        userids.add(ele)

total_pid = df['co-purchased'].values
pids=set()
for row in total_pid:
    for ele in row:
        pids.add(ele)
        
reviewed_products=set()
for row in df[['product_id','user_ids']].iterrows():
    if len(row[1][1]):
        reviewed_products.add(row[1][0])
print("The total number of products purchased in the data",len(df))       
print("The total number of users in the data:",len(userids))
print("the total number of product who has received atleaset one review:",len(reviewed_products))
print("The total number of co purchased products in the data:",len(pids))       
print("The toal number of products including(co purchased products) available in the data:",len(set(df['product_id']).difference(pids))+len(pids))
temp_pp = df['co-purchased'].apply(lambda x : len(x))
print("The total number of pair of products in the data:",temp_pp.sum())
print("The average number  co purchasing products per product:",temp_pp.mean())
temp_rev = df['review_ratings_list'].apply(lambda x : len(x))
print("The total number of reviews in the data:",temp_rev.sum())
print("The average number of reviews per product in the data:",temp_rev.mean())
print("The average rating per product in the data:",df['avg_rating'].mean())

The total number of products purchased in the data 548552
The total number of users in the data: 1555170
the total number of product who has received atleaset one review: 402724
The total number of co purchased products in the data: 410057
The toal number of products including(co purchased products) available in the data: 721342
The total number of pair of products in the data: 1788725
The average number  co purchasing products per product: 3.260812101678601
The total number of reviews in the data: 7593244
The average number of reviews per product in the data: 13.842341291254066
The average rating per product in the data: 3.1752012571278567
Wall time: 32.2 s


In [7]:
data = df.copy()

##### preparing data for collaborative filtering rating matrix

#### getting the data frame for product group=book
#### Because of the memory issue
* applying filters: each product should have atleast 100 reviews
* applying filters: each user should have given atleast 10 review

In [8]:
df_sub = df.loc[(df['num_reviews']>10)&(df['group']=='book')]
df_sub = df_sub[['product_id','user_ids','review_ratings_list','num_reviews']]
df_sub.shape

(63296, 4)

In [9]:
%%time
filter_userid=list()
for  il in df_sub['user_ids']:
    for item in il:
        filter_userid.append(item)  
count_dict = Counter(filter_userid)
filter_userid = set()
for k,v in count_dict.items():
    if v>=10:
        filter_userid.add(k)
filter_pid = df_sub['product_id'].unique()
print("filtered_user_ids:",len(filter_userid))
print("filtered_product_ids:",len(filter_pid))

filtered_user_ids: 52914
filtered_product_ids: 63296
Wall time: 1.48 s


In [10]:
final_list=[]
for row in df_sub.iterrows():
    prod = row[1][0]
    for idx,user in enumerate(row[1][1]):
        if user in (filter_userid):
            rating = row[1][2][idx]
            final_list.append((user,prod,rating))          

In [11]:
ratings=pd.DataFrame(final_list,columns=['userID','productID','Rating'])

In [12]:
ratings['Rating'].value_counts(normalize=1)*100

5    58.293704
4    20.492759
3     9.402784
1     6.337630
2     5.473122
Name: Rating, dtype: float64

In [13]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2163774 entries, 0 to 2163773
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   userID     object
 1   productID  object
 2   Rating     int64 
dtypes: int64(1), object(2)
memory usage: 49.5+ MB


In [14]:
ratings.to_csv('ratings_book.csv',index=False)

#### getting the data frame for product group=music
#### Because of the memory issue
* applying filters: each product should have atleast 50 reviews
* applying filters: each user should have given atleast 10 review

In [15]:
df_sub = df.loc[(df['num_reviews']>10)&(df['group']=='music')]
df_sub = df_sub[['product_id','user_ids','review_ratings_list','num_reviews']]
df_sub.shape

(22869, 4)

In [16]:
%%time
filter_userid=list()
for  il in df_sub['user_ids']:
    for item in il:
        filter_userid.append(item)  
count_dict = Counter(filter_userid)
filter_userid = set()
for k,v in count_dict.items():
    if v>=10:
        filter_userid.add(k)
filter_pid = df_sub['product_id'].unique()
print("filtered_user_ids:",len(filter_userid))
print("filtered_product_ids:",len(filter_pid))

filtered_user_ids: 12827
filtered_product_ids: 22869
Wall time: 529 ms


In [17]:
final_list=[]
for row in df_sub.iterrows():
    prod = row[1][0]
    for idx,user in enumerate(row[1][1]):
        if user in (filter_userid):
            rating = row[1][2][idx]
            final_list.append((user,prod,rating))          

In [18]:
ratings_music=pd.DataFrame(final_list,columns=['userID','productID','Rating'])

In [19]:
ratings_music.to_csv('ratings_music.csv',index=False)